<a href="https://colab.research.google.com/github/Illumaria/iti_funds_nav/blob/master/iti_ruse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import re
import pandas

In [0]:
#url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/ETFs/company-news.html?fourWayKey=LU1483649312LUUSDEUET' # RUSE initial link
#url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/ETFs/company-news.html?fourWayKey=LU1483649825LUUSDEUET' # RUSB initial link
#url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/rns/company-news.html?tidm=RUSE&isin=LU1483649312&newsType=' # RUSE edited link; USE THIS ONE FOR RUSE
url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/rns/company-news.html?tidm=RUSB&isin=LU1483649825&newsType=' # RUSB edited link; USE THIS ONE FOR RUSB

In [0]:
iti_resp = requests.get(url)

In [0]:
bs = BeautifulSoup(iti_resp.text)

In [0]:
# Get the number of pages:
pages = bs.find('p', text = re.compile('Page ')).text
num_pages = int(pages.split()[3])

In [0]:
# Uncomment the following code if we need to take into account the NAV publication dates
# instead of their calculation dates (code update required):
#dates = bs.find_all('span', class_="hour")
#for date in dates:
#  print(date.text.split()[0]+" "+date.text.split()[1]+" "+date.text.split()[2])

In [0]:
# Find all the partial links on the first page:
scripts = bs.find_all('a', text = re.compile('Net Asset Value'))

In [0]:
# Get full links:
urls = []
urls.extend(['https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1] for i in range(len(scripts))])

In [0]:
# Do the same for other pages:
for i in range(2,num_pages+1):
  resp = requests.get('{}{}{}'.format(url, "&page=",i))
  bs = BeautifulSoup(resp.text)
  scripts = bs.find_all('a', text = re.compile('Net Asset Value'))
  urls.extend(['https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1] for i in range(len(scripts))])

In [0]:
# Get the number of urls to determine the number of table rows:
nrows = len(urls)
# Create an empty pandas dataframe with 0 to (nrows-1) rows and 0 to (2-1) columns:
table = pandas.DataFrame(columns=range(0,4), index = range(0, nrows))

In [0]:
# Fill the dataframe with data:
for row in range(nrows):
  # This is going to take some time... Uncomment to see the "progress bar" :D
  #print(url)
  resp = requests.get(urls[row])
  bs = BeautifulSoup(resp.text)

  date = bs.find('p', text = re.compile('DEALING DATE')).text
  table.iat[row,0] = date.split()[2]

  nav = bs.find('p', text = re.compile('NAV PER SHARE')).text
  table.iat[row,1] = float(nav.split()[4])

  shares = bs.find('p', text = re.compile('NUMBER OF SHARES')).text
  table.iat[row,2] = int(shares.split()[5])

  table.iat[row,3] = fnav*ishares

In [14]:
# Rename the columns:
table.columns = ['Date', 'NAV per share', 'Number of shares', 'Total NAV']
table

,Date,NAV per share,Number of shares,Total NAV
0,19-Feb-2020,28.3771,355000,1.00739e+07
1,18-Feb-2020,28.3693,355000,1.00711e+07
2,17-Feb-2020,28.3463,355000,1.00629e+07
3,14-Feb-2020,28.3353,355000,1.0059e+07
4,13-Feb-2020,28.292,355000,1.00437e+07
...,...,...,...,...
297,05-Dec-2018,24.4668,203000,4.96676e+06
298,04-Dec-2018,24.4693,203000,4.96727e+06
299,03-Dec-2018,24.4391,203000,4.96114e+06
300,30-Nov-2018,24.3679,203000,4.94668e+06


In [15]:
# Set the date column to datetime type:
table['Date'] = pandas.to_datetime(table['Date'], dayfirst=True)
table

,Date,NAV per share,Number of shares,Total NAV
0,2020-02-19,28.3771,355000,1.00739e+07
1,2020-02-18,28.3693,355000,1.00711e+07
2,2020-02-17,28.3463,355000,1.00629e+07
3,2020-02-14,28.3353,355000,1.0059e+07
4,2020-02-13,28.292,355000,1.00437e+07
...,...,...,...,...
297,2018-12-05,24.4668,203000,4.96676e+06
298,2018-12-04,24.4693,203000,4.96727e+06
299,2018-12-03,24.4391,203000,4.96114e+06
300,2018-11-30,24.3679,203000,4.94668e+06


In [16]:
# Do some type conversions:
table['NAV per share'] = pandas.to_numeric(table['NAV per share'])
table['Number of shares'] = pandas.to_numeric(table['Number of shares'])
table['Total NAV'] = pandas.to_numeric(table['Total NAV'])
# Change the index column:
table.index = pandas.DatetimeIndex(table['Date'])
table

,Date,NAV per share,Number of shares,Total NAV
Date,,,,
2020-02-19,2020-02-19,28.3771,355000,10073870.5
2020-02-18,2020-02-18,28.3693,355000,10071101.5
2020-02-17,2020-02-17,28.3463,355000,10062936.5
2020-02-14,2020-02-14,28.3353,355000,10059031.5
2020-02-13,2020-02-13,28.2920,355000,10043660.0
...,...,...,...,...
2018-12-05,2018-12-05,24.4668,203000,4966760.4
2018-12-04,2018-12-04,24.4693,203000,4967267.9
2018-12-03,2018-12-03,24.4391,203000,4961137.3


In [17]:
# Get rid of the data column since it copies the index column now:
table = table.drop(columns=['Date']).dropna()
table

,NAV per share,Number of shares,Total NAV
Date,,,
2020-02-19,28.3771,355000,10073870.5
2020-02-18,28.3693,355000,10071101.5
2020-02-17,28.3463,355000,10062936.5
2020-02-14,28.3353,355000,10059031.5
2020-02-13,28.2920,355000,10043660.0
...,...,...,...
2018-12-05,24.4668,203000,4966760.4
2018-12-04,24.4693,203000,4967267.9
2018-12-03,24.4391,203000,4961137.3


In [18]:
# Inverse the dataframe:
table = table.iloc[::-1]
table

,NAV per share,Number of shares,Total NAV
Date,,,
2018-11-29,24.3559,203000,4944247.7
2018-11-30,24.3679,203000,4946683.7
2018-12-03,24.4391,203000,4961137.3
2018-12-04,24.4693,203000,4967267.9
2018-12-05,24.4668,203000,4966760.4
...,...,...,...
2020-02-13,28.2920,355000,10043660.0
2020-02-14,28.3353,355000,10059031.5
2020-02-17,28.3463,355000,10062936.5
